In [1]:
import numpy as np 
import random
import hdnet.hopfield as hdn
import os
from scipy.sparse import csr_matrix, save_npz, load_npz
import scipy.io as spio

class ModifiedHopfieldNet():
    
    def __init__(self, N, in_directory, out_directory=None, splits=3, train_percent=0.66, num_nets=5, exp_type='J'):
        self.N = N
        self.in_directory = in_directory
        if out_directory is None:
            self.out_directory = self.in_directory
        else:
            self.out_directory = out_directory
        self.splits = splits
        self.experiments = []
        self.train_percent = train_percent
        self.num_nets = num_nets
        self.type = exp_type
        self.networks = []
        self.load_and_save_data()
        
    def load_and_save_data(self):
        for file in os.listdir(self.in_directory):
            filename = file[:-4] + '_sparse.npz'
            dt = .1 # can change this
            if filename not in os.listdir(self.out_directory):
                print(f'---------------------- Importing .mat file: {file} ----------------------')
                dat = spio.loadmat(os.path.join(self.in_directory, file))
                if self.type == 'map':
                    xs = []
                    StimTimes = dat['StimTimes']
                    Cs = np.array(dat['Cs'], dtype='uint32')
                    Ts = np.array(dat['Ts'], dtype='uint32')
                    foo_s = np.asarray([int(i/dt) for i in StimTimes])
                else:
                    Cs = dat['data']['Cs']
                    Ts = dat['data']['Ts']
                    Cs = np.array([a[0] for a in Cs.tolist()[0][0].tolist()], dtype='uint8')
                    Ts = np.array([a[0] for a in Ts.tolist()[0][0].tolist()], dtype='uint32')
                
                Tmax = np.max(Ts)
                foo_x = np.asarray([int(i/0.1) for i in Ts])
                
                ys = []
                for i in range(int(Tmax/dt)):
                    if i in foo_x:
                        # which neurons are firing
                        inds = (i*dt<Ts)*(Ts<(i+1)*dt)
                        neurons = Cs[inds]
                        foo2 = np.zeros(self.N)
                        foo2[neurons] = 1
                    else:
                        foo2 = np.zeros(self.N)
                    # is the stimulus firing
                    if self.type == 'map':
                        if i in foo_s:
                            foo2[-1] = 1
                        else:
                            foo2[-1] = 0
                    ys.append(foo2)

                ys = np.asarray(ys, dtype='uint8').squeeze()
                self.experiments.append(ys)
                
                y_sparse = csr_matrix(ys, dtype='uint8')
                save_npz(os.path.join(self.out_directory, filename), y_sparse)
            else:
                print(f'---------------------- Loading file: {filename} ----------------------')
                ys = load_npz(os.path.join(self.out_directory, filename)).toarray()
                self.experiments.append(ys)
                
    def build_and_train_networks(self):
        for i, memories in enumerate(self.experiments):
            print(f'---------------------- Conducting experiment: {i} ----------------------')
            chunks = []
            memories_chunked = self.chunked(memories, self.splits)
            for j, memory_chunk in enumerate(memories_chunked):
                avg_acc = []
                chunked_nets = []
                for _ in range(self.num_nets):
                    hop = hdn.HopfieldNetMPF(N=self.N)
                    rand_memories = np.array([random.choice(memory_chunk) for _ in range(round(len(memory_chunk)*self.train_percent))])
                    patterns = hop.store_patterns_using_mpf(rand_memories)
                    avg_acc.append(self.get_accuracy(memory_chunk,hop))
                    chunked_nets.append(hop)
                self.networks.append(chunked_nets)
                print(f'Experiment: {i} // Chunk: {j} // Avg Accuracy: {round(np.mean(avg_acc),3)} +/- {round(np.std(avg_acc),3)}')   
            print(f'---------------------- Finished experiment: {i} ----------------------')

    def chunked(self, iterable, n):
        chunksize = int(math.ceil(len(iterable) / n))
        return (iterable[i * chunksize:i * chunksize + chunksize]
                for i in range(n))
            
    def getL(self, x, h1, A):
        L = np.exp(np.dot(h1.T,x)+A)
        if L > 1:
            return 1
        else:
            return 0
    
    def get_preds(self, y, hop):
        J = hop.J
        h = -hop.theta
        A = J[-1,-1]
        B = h[-1]
        # J0 = J[:-1, :-1]
        j = J[-1, :-1]
        # jT = J[-1, :-1]
        # J = J0
        h1 = 2*j
        # h0 = h
        A = A+B
        x = y[:-1]
        p = self.getL(x, h1, A)
        return p
    
    def get_accuracy(self, memories, hop):
        accuracy = 0
        y_preds = []
        y_true = []
        for k, i in enumerate(memories):
            y_preds.append(self.get_preds(i, hop))
            y_true.append(i[-1])
            if y_preds[k] == y_true[k]:
                accuracy += 1
        accuracy = accuracy / len(memories)
        return round(accuracy*100, 3)
    def get_js(self):
        
        for 
    
    


SyntaxError: invalid syntax (<ipython-input-1-baae88f67fa5>, line 133)

In [117]:
net = ModifiedHopfieldNet(N=61, in_directory='toy_data/', out_directory='new_data/', exp_type='map')
net.build_and_train_networks()

---------------------- Importing .mat file: Expt1.mat ----------------------
---------------------- Conducting experiment: 0 ----------------------
Experiment: 0 // Chunk: 0 // Avg Accuracy: 98.678 +/- 0.0
Experiment: 0 // Chunk: 1 // Avg Accuracy: 99.152 +/- 0.0
Experiment: 0 // Chunk: 2 // Avg Accuracy: 98.685 +/- 0.0
---------------------- Finished experiment: 0 ----------------------


In [2]:
# Train hopfield network on concatenated samples
with open('memories.pkl', 'rb') as file:
   memories = pickle.load(file)
memories = np.array(memories)
# hop = hdn.HopfieldNetMPF(N=61)
# patterns = hop.store_patterns_using_mpf(memories)
# hop.bits_recalled(memories)
# with open('hopfieldNet.pkl', 'wb') as file:
#    pickle.dump(hop, file)

In [69]:
# Divide experiment in three sections and compute accuracy for each half
import random
beginning = memories[:len(memories)//3, :]
middle = memories[len(memories)//3:2*len(memories)//3, :]
ending = memories[2*len(memories)//3:, :]
data = [beginning, middle, ending]
hops = [hdn.HopfieldNetMPF(N=61)]*3
for k, i in enumerate(data):
    avg_acc = []
    for _ in range(5):
        hop = hdn.HopfieldNetMPF(N=61)
        patterns = hop.store_patterns_using_mpf([random.choice(i) for _ in range(round(len(i)*4/5))])
        avg_acc.append(get_accuracy(data[k],hop))
    print(f'Avg Accuracy: {round(np.mean(avg_acc),3)} +/- {round(np.std(avg_acc),3)}')

Avg Accuracy: 98.24199999999999 +/- 0.11822013364905325
Avg Accuracy: 98.82 +/- 0.063874877690685
Avg Accuracy: 98.502 +/- 0.011661903789692181


In [90]:
net.networks

[]

In [4]:
beginning = memories[:len(memories)//2, :]
ending = memories[len(memories)//2:, :]
data = [beginning, ending]
hops = [hdn.HopfieldNetMPF(N=61), hdn.HopfieldNetMPF(N=61)]
acc = []
for k, i in enumerate(data):
    patterns = hops[k].store_patterns_using_mpf(i)
#     print(f'Bits Recalled: {hops[k].bits_recalled(i)}')
    acc.append(get_accuracy(data[k],hops[k]))
    print(f'Accuracy: {acc[k]}')

Accuracy: 99.07
Accuracy: 98.55


In [27]:
from __future__ import division

memories = net.experiments[0][:10000, :]

import math

def chunked(iterable, n):
    chunksize = int(math.ceil(len(iterable) / n))
    return (iterable[i * chunksize:i * chunksize + chunksize]
            for i in range(n))
list1 = memories.tolist()
len(list(chunked(list1, 3))[2])

3332

In [14]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_preds)
print(precision, recall)
print(confusion_matrix(y_true, y_preds))

NameError: name 'y_true' is not defined

In [3]:
import pickle
with open('hopfieldNet.pkl', 'rb') as file:
   hop = pickle.load(file)